In [1]:
import os
import numpy as np
import pandas as pd
import openpyxl
import re

In [2]:
!pip install openpyxl

DEPRECATION: Loading egg at c:\users\adem mcharek\fruits\build\tfod\lib\site-packages\apache_beam-2.47.0rc1-py3.11-win-amd64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\adem mcharek\fruits\build\tfod\lib\site-packages\avro_python3-1.10.2-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\adem mcharek\fruits\build\tfod\lib\site-packages\contextlib2-21.6.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\adem mcharek\fruits\build

In [3]:
filepath = '2-17-24.csv'
data = pd.read_csv(filepath)
data.head()

,Company,Role,Location,Application/Link,Date Posted
0,**[Zoox](https://simplify.jobs/c/Zoox)**,Perception Intern,"San Mateo, CA",<a href=https://jobs.lever.co/zoox/879af812-dd...,Feb 16
1,**[Wisk](https://simplify.jobs/c/wisk-aero)**,Software Development Intern / Stagiaire en dév...,"Montreal, QC, Canada",<a href=https://jobs.lever.co/wisk/65d5c693-97...,Feb 16
2,↳,Software Intern,"Mountain View, CA",<a href=https://jobs.lever.co/wisk/5cfc99ef-2e...,Feb 16
3,↳,Information Technology Intern,"Mountain View, CA",<a href=https://jobs.lever.co/wisk/d9a2231b-7e...,Feb 16
4,↳,BMS Software Engineering Intern,"Mountain View, CA",<a href=https://jobs.lever.co/wisk/d96c8ba0-eb...,Feb 16


In [4]:
data['Company'] = data.Company.str.extract(r'\[(.*?)\]', expand=False)

data.head()

,Company,Role,Location,Application/Link,Date Posted
0,Zoox,Perception Intern,"San Mateo, CA",<a href=https://jobs.lever.co/zoox/879af812-dd...,Feb 16
1,Wisk,Software Development Intern / Stagiaire en dév...,"Montreal, QC, Canada",<a href=https://jobs.lever.co/wisk/65d5c693-97...,Feb 16
2,NaN,Software Intern,"Mountain View, CA",<a href=https://jobs.lever.co/wisk/5cfc99ef-2e...,Feb 16
3,NaN,Information Technology Intern,"Mountain View, CA",<a href=https://jobs.lever.co/wisk/d9a2231b-7e...,Feb 16
4,NaN,BMS Software Engineering Intern,"Mountain View, CA",<a href=https://jobs.lever.co/wisk/d96c8ba0-eb...,Feb 16


In [5]:
data['Application'] = data['Application/Link'].str.extract(r'<a(.*?)a>', expand=False)
data['Application'] = data['Application'].str.replace('href=', '')
data['Application'] = data['Application'].str.extract(r'(https.*Simplify)', expand=False)
data['Application']

0      https://jobs.lever.co/zoox/879af812-dd13-4113-...
1      https://jobs.lever.co/wisk/65d5c693-9701-4d64-...
2      https://jobs.lever.co/wisk/5cfc99ef-2e2c-4923-...
3      https://jobs.lever.co/wisk/d9a2231b-7e84-4742-...
4      https://jobs.lever.co/wisk/d96c8ba0-eb19-4315-...
                             ...                        
271    https://www.amazon.jobs/en/jobs/2495700/jr-sof...
272    https://matroid.breezy.hr/p/bf4668116295/apply...
273    https://boards.greenhouse.io/gainternships/job...
274    https://boards.greenhouse.io/drweng/jobs/55802...
275    https://boseallaboutme.wd1.myworkdayjobs.com/e...
Name: Application, Length: 276, dtype: object

In [6]:
data['Application'][1]

'https://jobs.lever.co/wisk/65d5c693-9701-4d64-be58-dde85cb2cc03/apply?utm_source=Simplify&ref=Simplify'

In [7]:
data['link'] = data['Application/Link'].str.extract(r'(https://simplify.*)')
data['link'] = data['link'].str.replace(r'".*', '"')
data['link'] = data['link'].str.extract(r'(https://[^"\s>]+)', expand=False) 
data['link'][1]

'https://simplify.jobs/p/b6993be6-b455-448e-b240-421054b25354?utm_source=GHList'

In [8]:
data['link'][1]

'https://simplify.jobs/p/b6993be6-b455-448e-b240-421054b25354?utm_source=GHList'

In [9]:

mask = data['Location'].str.startswith('<')
data.loc[mask, 'Location'] = data.loc[mask, 'Location'].str.extract(r'<\*\*(.*?)\*\*')


In [10]:
data = data.drop('Application/Link', axis=1)


In [11]:
data.head()

,Company,Role,Location,Date Posted,Application,link
0,Zoox,Perception Intern,"San Mateo, CA",Feb 16,https://jobs.lever.co/zoox/879af812-dd13-4113-...,https://simplify.jobs/p/1bd68e90-dac9-441c-877...
1,Wisk,Software Development Intern / Stagiaire en dév...,"Montreal, QC, Canada",Feb 16,https://jobs.lever.co/wisk/65d5c693-9701-4d64-...,https://simplify.jobs/p/b6993be6-b455-448e-b24...
2,NaN,Software Intern,"Mountain View, CA",Feb 16,https://jobs.lever.co/wisk/5cfc99ef-2e2c-4923-...,https://simplify.jobs/p/7bef5785-ef48-495f-a70...
3,NaN,Information Technology Intern,"Mountain View, CA",Feb 16,https://jobs.lever.co/wisk/d9a2231b-7e84-4742-...,https://simplify.jobs/p/460f6e00-98a2-4318-95b...
4,NaN,BMS Software Engineering Intern,"Mountain View, CA",Feb 16,https://jobs.lever.co/wisk/d96c8ba0-eb19-4315-...,https://simplify.jobs/p/efc1378d-f617-44bb-8f1...


In [12]:
data['Company'].fillna(method='ffill', inplace=True)

In [14]:
data['link'].fillna(method='ffill', inplace=True)

In [17]:
data.rename(columns={'Date Posted': 'Date'}, inplace=True)


In [18]:
data.head()

,Company,Role,Location,date,Application,link
0,Zoox,Perception Intern,"San Mateo, CA",Feb 16,https://jobs.lever.co/zoox/879af812-dd13-4113-...,https://simplify.jobs/p/1bd68e90-dac9-441c-877...
1,Wisk,Software Development Intern / Stagiaire en dév...,"Montreal, QC, Canada",Feb 16,https://jobs.lever.co/wisk/65d5c693-9701-4d64-...,https://simplify.jobs/p/b6993be6-b455-448e-b24...
2,Wisk,Software Intern,"Mountain View, CA",Feb 16,https://jobs.lever.co/wisk/5cfc99ef-2e2c-4923-...,https://simplify.jobs/p/7bef5785-ef48-495f-a70...
3,Wisk,Information Technology Intern,"Mountain View, CA",Feb 16,https://jobs.lever.co/wisk/d9a2231b-7e84-4742-...,https://simplify.jobs/p/460f6e00-98a2-4318-95b...
4,Wisk,BMS Software Engineering Intern,"Mountain View, CA",Feb 16,https://jobs.lever.co/wisk/d96c8ba0-eb19-4315-...,https://simplify.jobs/p/efc1378d-f617-44bb-8f1...


In [15]:
data.to_excel('2-17-24.xlsx', index=False)


In [19]:
data.to_csv('2-17-24-clean.csv', index=False)
